https://medium.com/tensorflow/building-a-text-classification-model-with-tensorflow-hub-and-estimators-3169e7aa568

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

#from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

In [2]:
#import pymongo
#mongoClient = pymongo.MongoClient("mongodb://localhost:27017/")
#db = mongoClient["textrecognition"]
#cursor = db["ready"].find({}, {'_id': False})
#df = pd.DataFrame(list(cursor))

df = pd.read_json("singlelabel2.json", lines=True)

In [3]:
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
df.head(10)

,_id,categories,text
0,{'$oid': '5cdbc77cc201f803ecf35fe3'},Spook,I was 10 years old when this happened to me. I...
1,{'$oid': '5cdbc77cc201f803ecf3415e'},Story,I'm Sorry I'm sorry I can't control myself. I'...
2,{'$oid': '5cdbc77cc201f803ecf35c58'},Story,I work as an engineer for a small TV station. ...
3,{'$oid': '5cdbc77cc201f803ecf363e3'},Joke,Vodka isn't the answer to all of my problems.....
4,{'$oid': '5cdbc77cc201f803ecf32259'},News,WASHINGTON â€” The economic fallout from Brita...
5,{'$oid': '5cdbc77cc201f803ecf326ff'},News,BRASÃ�LIA â€” Less than two months before Braz...
6,{'$oid': '5cdbc77cc201f803ecf36474'},Joke,Why did the monkey fall out of the tree? Cause...
7,{'$oid': '5cdbc77cc201f803ecf31fa9'},Tweet,getting home at 4 waking up at 9 :):)
8,{'$oid': '5cdbc77cc201f803ecf331b6'},News,Donald J. Trump has been working to mend fence...
9,{'$oid': '5cdbc77cc201f803ecf3549c'},Joke,"A plane is about to crash And suddenly, a woma..."


In [5]:
text = df['text']
categories = df['categories']

set1 = categories.apply(lambda x: x[0])
set2 = categories.apply(lambda x: x[1])

set1 = set1.unique()
set2 = set2.unique()

unique_categories = []

for s in set1:
    unique_categories.append(s)
for s in set2:
    unique_categories.append(s)

unique_categories

In [6]:
unique_categories = categories.unique()
unique_categories

array(['Spook', 'Story', 'Joke', 'News', 'Tweet'], dtype=object)

In [7]:
train_size = int(len(text) * .8)

train_text = text[:train_size]
train_categories = categories[:train_size]

test_text = text[train_size:]
test_categories = categories[train_size:]

In [8]:
text_embeddings = hub.text_embedding_column(
  "pieces_of_text", 
  #module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
  module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
  trainable=True
)

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
text_embeddings = elmo(
    "pieces_of_text",
    signature="default",
    as_dict=True)["elmo"]

In [9]:
#encoder = MultiLabelBinarizer()
#encoder = LabelBinarizer()
encoder = LabelEncoder()
encoder.fit_transform(categories)
train_encoded = encoder.transform(train_categories)
test_encoded = encoder.transform(test_categories)
num_classes = len(encoder.classes_)

In [10]:
encoder.classes_

array(['Joke', 'News', 'Spook', 'Story', 'Tweet'], dtype=object)

In [11]:
encoder.classes_.size

5

In [12]:
#https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2

optimizer = []
optimizer.append(tf.train.AdadeltaOptimizer(learning_rate=1))

In [13]:
optimizer

multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

In [14]:
estimators = []

for op in optimizer:
    estimators.append(tf.estimator.DNNClassifier(
        feature_columns = [text_embeddings],
        hidden_units=[32, 16],
        n_classes=encoder.classes_.size,
        optimizer=op
    ))

INFO:tensorflow:Using default config.


I0606 12:52:44.144406  1832 estimator.py:1739] Using default config.


W0606 12:52:44.155376  1832 estimator.py:1760] Using temporary folder as model directory: C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Bruger\\AppData\\Local\\Temp\\tmph7958z8q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029DEA0C7978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0606 12:52:44.163354  1832 estimator.py:201] Using config: {'_model_dir': 'C:\\Users\\Bruger\\AppData\\Local\\Temp\\tmph7958z8q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029DEA0C7978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


estimator = tf.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,16],
    feature_columns=[text_embeddings]
)

In [15]:
# Format the data for the numpy_input_fn
features = {
  "pieces_of_text": np.array(train_text)
}
labels = np.array(train_encoded)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=False, 
    batch_size=32, 
    num_epochs=3
)

In [16]:
for estimator in estimators:
    estimator.train(input_fn=train_input_fn)

Instructions for updating:
Colocations handled automatically by placer.


W0606 12:52:44.250175  1832 deprecation.py:323] From C:\Users\Bruger\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0606 12:52:44.280042  1832 deprecation.py:323] From C:\Users\Bruger\Anaconda3\lib\site-packages\tensorflow_estimator\python\estimator\inputs\queues\feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

I0606 12:55:11.187667  1832 basic_session_run_hooks.py:247] loss = 2.058451, step = 2501 (5.302 sec)


INFO:tensorflow:Saving checkpoints for 2593 into C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q\model.ckpt.


I0606 12:55:15.956918  1832 basic_session_run_hooks.py:594] Saving checkpoints for 2593 into C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q\model.ckpt.


INFO:tensorflow:Loss for final step: 0.65870035.


I0606 12:55:29.930908  1832 estimator.py:359] Loss for final step: 0.65870035.


In [17]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"pieces_of_text": np.array(test_text).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)

In [18]:
estimators[0].evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


I0606 12:55:31.243371  1832 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 12:55:31.306203  1832 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 2593: accuracy = 0.92421174, average_loss = 0.21328875, global_step = 2593, loss = 26.812336


I0606 12:55:42.131534  1832 estimator.py:1979] Saving dict for global step 2593: accuracy = 0.92421174, average_loss = 0.21328875, global_step = 2593, loss = 26.812336


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2593: C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q\model.ckpt-2593


I0606 12:55:42.372889  1832 estimator.py:2039] Saving 'checkpoint_path' summary for global step 2593: C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q\model.ckpt-2593


{'accuracy': 0.92421174,
 'average_loss': 0.21328875,
 'loss': 26.812336,
 'global_step': 2593}

In [20]:
estimators[0].evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.


I0606 12:56:04.734031  1832 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 12:56:04.774886  1832 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 2593: accuracy = 0.92421174, average_loss = 0.21328875, global_step = 2593, loss = 26.812336


I0606 12:56:14.582609  1832 estimator.py:1979] Saving dict for global step 2593: accuracy = 0.92421174, average_loss = 0.21328875, global_step = 2593, loss = 26.812336


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2593: C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q\model.ckpt-2593


I0606 12:56:14.647356  1832 estimator.py:2039] Saving 'checkpoint_path' summary for global step 2593: C:\Users\Bruger\AppData\Local\Temp\tmph7958z8q\model.ckpt-2593


{'accuracy': 0.92421174,
 'average_loss': 0.21328875,
 'loss': 26.812336,
 'global_step': 2593}

estimators[2].evaluate(input_fn=eval_input_fn)

estimators[3].evaluate(input_fn=eval_input_fn)

In [45]:
from os import listdir
from os.path import isfile, join
mypath = 'C:/Users/Bruger/BigDataPython/TensorDNN/Samples'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

raw_data = []
for f in onlyfiles:
    print(f)
    with open(mypath + '/' + f, 'r', encoding="utf-8") as file:
        data = file.read().replace('\n', ' ')
        raw_data.append(data)

News.txt
News2.txt
Twitter.txt


In [46]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"pieces_of_text": np.array(raw_data).astype(np.str)}, shuffle=False)

results = []
for estimator in estimators:
    results.append(estimator.predict(predict_input_fn))

In [47]:
for result in results:
    for categories in result:
        top = categories['probabilities'].argsort()[-2:][::-1]
        print()
        for category in top:
            text_category = encoder.classes_[category]
            print(text_category + ': ' + str(round(categories['probabilities'][category] * 100, 2)) + '%', end=" | ")
    print()

INFO:tensorflow:Calling model_fn.


I0606 13:26:04.350225  1832 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0606 13:26:04.389122  1832 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Running local_init_op.


I0606 13:26:06.406912  1832 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0606 13:26:07.572590  1832 session_manager.py:493] Done running local_init_op.



Story: 48.13% | News: 45.78% | 
News: 100.0% | Joke: 0.0% | 
Joke: 44.31% | Story: 28.16% | 
